forward 함수가 특정 데이터 구조에 의존하는 문제를 해결하기 위해 작동 방식을 분석해보는 파일입니다.

## 0. forward 함수의 구조

In [1]:
# jupyter notebook에서 import 해서 쓰는 모듈의 코드가 변경될 시, 변동 사항을 자동으로 반영해주는 기능 켜기
%load_ext autoreload
%autoreload 2

In [1]:
import os
from os import path
import sys

PROJECT_DIR = path.abspath(path.join(os.getcwd(), "..", ".."))
sys.path.append(PROJECT_DIR)

import torch
import torch.nn as nn
from torch import Tensor
from utils.news_viewer import NewsViewer


# model.forward 가져와보기
def forward(batch, news_encoder, user_encoder):
    # encode candidate news
    candidate_news_vector = news_encoder(batch["c_title"])
    # encode history 
    clicked_news_vector = news_encoder(batch["h_title"])
    # encode user
    user_vector = user_encoder(clicked_news_vector)
    # compute scores for each candidate news
    clicks_score = torch.bmm(
        candidate_news_vector,
        user_vector.unsqueeze(dim=-1)).squeeze(dim=-1)
    
    return clicks_score

In [2]:
news_viewer = NewsViewer(
    path.join(PROJECT_DIR, "data", "MIND", "demo", "test", "news.tsv"),
    path.join(PROJECT_DIR, "data", "preprocessed_data", "demo", "test", "news2int.tsv")
)

In [3]:
news_viewer.news.head()

,category,subcategory,title,abstract,url,title_entities,abstract_entities
news_id,,,,,,,
N3112,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://www.msn.com/en-us/lifestyle/lifestyler...,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
N10399,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://www.msn.com/en-us/news/world/the-cost-...,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
N12103,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://www.msn.com/en-us/health/voices/i-was-...,[],"[{""Label"": ""National Basketball Association"", ..."
N20460,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://www.msn.com/en-us/health/medical/how-t...,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."
N5409,weather,weathertopstories,It's been Orlando's hottest October ever so fa...,There won't be a chill down to your bones this...,https://www.msn.com/en-us/weather/weathertopst...,"[{""Label"": ""Orlando, Florida"", ""Type"": ""G"", ""W...","[{""Label"": ""Orlando, Florida"", ""Type"": ""G"", ""W..."


In [4]:
news_viewer.news2int.head()

,news_id
news_index,
1,N3112
2,N10399
3,N12103
4,N20460
5,N5409


In [5]:
news_viewer.show_news_by_index(10000)

News ID: N23737
Category: finance
SubCategory: financenews
Title: Colorado Cold Sends New Zealand Family To Goodwill: 'It Was, Like, Freezing'
Abstract: Snow and frigid temperatures in Colorado have made Goodwill Industries of Denver a busy place.


In [6]:
news_viewer.get_news_by_index(10000)

{'news_index': 10000,
 'news_id': 'N23737',
 'category': 'finance',
 'subcategory': 'financenews',
 'title': 'finance',
 'abstract': 'Snow and frigid temperatures in Colorado have made Goodwill Industries of Denver a busy place.'}

## 1. 테스트에 쓸 모델 불러오기

In [7]:
from utils.model_manager import ModelManager
from utils.base_manager import ManagerArgs

args = ManagerArgs(
    config_path = path.join(PROJECT_DIR, "config/model/nrms/exp_demo1.yaml"),
    test_ckpt_path = path.join(PROJECT_DIR, "logs/lightning_logs/checkpoints/nrms/exp_demo1/epoch=24-val_auc_epoch=0.6996.ckpt")
)
model_manager = ModelManager(PROJECT_DIR, args, "test")

c:\Users\USER\anaconda3\envs\newsrec\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Seed set to 1234
 23%|██▎       | 4262/18723 [00:01<00:03, 4120.64it/s]


KeyboardInterrupt: 

In [ ]:
batch_data = model_manager.get_batch_from_dataloader(0)
model_manager.show_batch_struct(batch_data)

{'user': tensor([629]), 'h_idxs': tensor([[    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0, 12104, 10930,  6825,  3194, 13519,  9108,  8822,  3699,  3946,
          3699,  5847, 14338,  4653,  3140,  2258,  9712, 13792,  1429,  4985,
          5399,  5070,  5882,  3875, 14187, 10398, 12319, 13335,  8722,  7729,
         10700,   691,  7970,  6370, 12074, 20989, 25448, 20184, 16747, 26145]],
       device='cuda:0'), 'h_title': tensor([[[    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
         [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
         [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
         [    0,     0,     0,     0,     0,     0

AttributeError: 'list' object has no attribute 'tolist'

## 현재까지 word embedding 레이어에 대해 알아낸 내용
1. nn.Embedding.from_pretrained() 함수는 모델의 word embedding 레이어를 생성한다.<br/>
2. 해당 레이어는 `freeze: bool` param에 따라 학습 대상에 포함시킬 수 있다.<br/>
    포함될 경우, 임베딩 레이어는 모델 학습시 가중치 업데이트의 대상이 된다.(즉 fine tuning을 진행한다.)<br/>
3. 여기서 말하는 가중치(weights)는 embeddings 인자로 넘겨준 pretrained_word_embedding에 저장된 모든 임베딩 벡터이다.<br/>
    (실제로 불러온 모든 임베딩 벡터는 word_embedding.weight를 통해 출력해볼 수 있다.)<br/>
    즉 from_pretrained()를 쓸 경우, 임베딩 레이어를 사전 학습된 임베딩 테이블을 기반으로 생성한 후 현재 모델에 맞게 fine tuning을 진행하는 것이다.<br/>
    -> 여기서 임베딩 벡터를 embedding weights라고 부르는 이유를 알 수 있는데, 실제로 모든 임베딩 벡터가 해당 레이어의 가중치(weights)이기 때문이다.<br/>
4. 따라서 ckpt파일을 생성할 때, 모든 임베딩 벡터 또한 해당 파일에 저장된다.<br/>
    즉 임베딩 벡터를 과도하게 불러올 시 모델이 상당히 무거워질 수 있다.<br/>
5. 여기서 알 수 있듯 임베딩 벡터는 임베딩 레이어에서 미리 저장된 내용을 꺼내오는 것에 불과하므로, 신규 뉴스 데이터에 새로운 토큰이 등장할 경우 해당 뉴스를 그냥 쓸 수는 없고 해당 토큰의 임베딩 가중치를 레이어에 추가해 줘야 한다.<br/>
    -> 물론 현재 레이어의 임베딩 벡터는 fine tune이 진행된 상태이기 때문에 기존의 임베딩 벡터와 의미 상의 호환이 잘 안될 가능성이 있지만, 신규 뉴스를 추천해주지 못하는 것 보다는 훨씬 나을 것이다.<br/>
6. 실제 서비스 상황에서는 신규 뉴스 데이터를 포함시켜서 모델을 일정 주기로 재학습시켜 업데이트할 것이므로, 5번 내용은 업데이트 사이의 땜빵용으로 쓰일 수도 있을 것이다.

In [17]:
"""
nn.embedding 레이어는 복수의 인덱스를 넣어 동시에 임베딩 벡터로 변환할 수 있습니다.
즉 아래와 같습니다.
(embedding_dim == 임베딩 벡터 길이)

1. 특정 인덱스의 임베딩 벡터 뽑기 (단일 title)
embedding(torch.tensor([2, 5, 7])) -> 입력의 shape는 (3)
출력 형태: (3, embedding_dim)

2. 여러 특정 인덱스의 임베딩 벡터 뽑기 (단일 history)
embedding(torch.tensor([[2, 5, 7], [3, 6, 8]])) -> 입력의 shape는 (2, 3)
출력 형태: (2, 3, embedding_dim)
"""

word_embedding = nn.Embedding.from_pretrained(
    model_manager.pretrained_word_embedding,
    freeze=model_manager.config.freeze_word_embeddings,
    padding_idx=0)

In [18]:
print(word_embedding.weight.shape) # 크기
print(word_embedding.weight.dtype) # 데이터 타입

torch.Size([42561, 300])
torch.float32


In [ ]:
# 1. word_embedding 레이어에서 1번 가중치를 가져옵니다.
vec: Tensor = word_embedding(torch.tensor([1]))
# 2. 해당 Tensor 데이터가 cpu, gpu중 어디에 있는지 확인합니다.
# 만약 gpu에 있다면, .numpy()로 변환하기 전에 .cpu()를 통해 위치를 옮겨야 합니다. 
print(vec.device)
# 3. 만약 해당 텐서가 파이토치의 학습 대상(vec.requires_grad == True)이라면, 변환 이전에 .detach()를 통해 해당 계산 그래프(autograd?)에서 분리해야 합니다.
print(vec.requires_grad)
# 4. 모든 요소를 포함한 코드
# 해당 코드는 위의 조건에 따라 달라질 수 있습니다.
vec.detach().cpu().numpy()

In [20]:
vec: Tensor = word_embedding(torch.tensor([1]))
vec.shape

torch.Size([1, 300])

## 핵심 기능 뽑아내기 
1. 뉴스 데이터(ex. 제목, 본문)를 임베딩 벡터로 변환한다. (변환 형태는 matrix)
2. 변환한 데이터를 통해 news vector를 생성한다.
3. history를 news vector로 변환후, 이를 통해 user vector를 생성한다.
4. 후보 뉴스를 변환한 news vector와 user vector의 내적을 통해 click score를 계산한다.

In [26]:
# 실험에 사용할 뉴스 데이터
candidate_news = [
    {
        "newsID": "N3112",
        "category": "lifestyle",
        "subcategory": "lifestyleroyals",
        "title": "The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By",
        "abstract": "Shop the notebooks, jackets, and more that the royals can't live without.",
        "label": 1
    },
    {
        "newsID": "N10399",
        "category": "news",
        "subcategory": "newsworld",
        "title": "The Cost of Trump's Aid Freeze in the Trenches of Ukraine's War",
        "abstract": "Lt. Ivan Molchanets peeked over a parapet of sand bags at the front line of the war in Ukraine. Next to him was an empty helmet propped up to trick snipers, already perforated with multiple holes.",
        "label": 0
    }
]

history_news = [
    {
        "newsID": "N12103",
        "category": "health",
        "subcategory": "voices",
        "title": "I Was An NBA Wife. Here's How It Affected My Mental Health.",
        "abstract": "I felt like I was a fraud, and being an NBA wife didn't help that. In fact, it nearly destroyed me."
    },
    {
        "newsID": "N20460",
        "category": "health",
        "subcategory": "medical",
        "title": "How to Get Rid of Skin Tags, According to a Dermatologist",
        "abstract": "They seem harmless, but there's a very good reason you shouldn't ignore them. The post How to Get Rid of Skin Tags, According to a Dermatologist appeared first on Reader's Digest."
    }
]

# NewsEncoder와 TimeDistributed 클래스의 동작 방식

## 입력 데이터
news encoder에 넣어줄 입력을 생성할 때 고려할 점은 다음과 같습니다.

1. 값은 `Tensor` 타입입니다.
2. 실제 모델의 레이어는 batch 형태의 데이터를 받으므로, 입력의 형식에 유의해야 합니다.
3. 해당 `Tensor`는 모델과 같은 device에 위치해야 합니다.

실제 batch의 title 데이터는 아래와 같은 형태입니다:

- shape => (64, 28, 20)
  - 64: batch size (고정)
  - 28: 뉴스 개수 (impression마다 다름)
  - 20: title의 token 개수 (고정)

하지만 `NewsEncoder` 클래스는 하나의 데이터에 포함된 뉴스들을 벡터로 변환합니다.
따라서 연산 최적화를 위해, 모델에서는 `news_encoder`를 여러 데이터를 한 번에 처리하는 `TimeDistributed`라는 wrapper 클래스로 감쌉니다.

## TimeDistributed 클래스의 역할

해당 클래스가 하는 일은 다음과 같습니다.  
만약 shape가 (batch_num, news_num, title_idxs_num) 형태인 데이터를 받으면,  
for문으로 하나씩 처리하는 대신 아래와 같이 진행합니다:

- 1. 입력 데이터를  
  `(batch_num * news_num, title_idxs_num)` 형태로 변환한 뒤,  
  각 뉴스의 title 인덱스들을 `news_encoder`에 넣습니다.

- 2. 이때 `title_idxs_num` 부분의 데이터는 인덱스로 표현된 title이며,  
  이 인덱스들이 실제 `news_vector`로 인코딩됩니다.  
  즉, 출력은 `(batch_num * news_vec_num, news_vec_dim)` 형태가 됩니다.  
  여기서 `news_num`과 `news_vec_num`은 사실상 동일한 기준의 값(뉴스 개수)이지만,
  `title_idxs_num`은 토큰 개수이고 `news_vec_dim`은 벡터의 차원입니다.

- 3. 마지막으로 이 출력을 다시  
  `(batch_num, news_num, news_vec_dim)` 형태로 reshape하여 최종 결과를 내보냅니다.

## 이런 방식이 가능한 이유

`nn.MultiheadAttention` 레이어의 입력 query는 기본적으로  
**(L, N, E)** 형태로 요구됩니다.  
- L = 시퀀스 길이 (토큰 개수)  
- N = 배치 사이즈 (여기서는 뉴스 개수)  
- E = 임베딩 차원

즉, `NewsEncoder`가 받아야 하는 데이터의 구성은 (뉴스 개수, token 개수)입니다.

따라서 NewsEncoder 내부의 흐름과 데이터 구조 변화는 다음과 같습니다:
  - num_news = 뉴스 개수  
  - seq_len = 토큰 개수  
  - emb_dim = 임베딩 벡터 길이

  1. 입력: `(num_news, seq_len)`  
  2. 임베딩 적용: `(num_news, seq_len, emb_dim)`  
  3. 차원 순서 변경 (permute): `(seq_len, num_news, emb_dim)`  
  4. MultiheadAttention 적용: 출력은 다시 `(num_news, seq_len, emb_dim)`  
  5. AdditiveAttention 적용: 최종적으로 `(num_news, news_vec_dim)`

(batch_num * news_num, title_idxs_num)형태의 변환은 배치 내부의 모든 뉴스를 한 차원으로 쫙 펼쳐서 넣어준다는 뜻입니다.
또한 MHSA와 AddictiveAttention 레이어에서는 각 뉴스에 대해 개별적으로 vector 변환을 진행하므로,  
이런 변환 방식이 최종 결과에 영향을 미치지 않으며 더 빠릅니다.

In [23]:
from nltk.tokenize import word_tokenize
from pandas import DataFrame
from models.nrms import NRMS
import pytorch_lightning as pl
from torch.nn.utils.rnn import pad_sequence

def get_news_vector(news_list: list, model: pl.LightningModule, word2int: DataFrame) -> Tensor:
    # 모델의 device를 구합니다.
    device = next(model.parameters()).device
    
    title_idxs = []
    for news in news_list:
        """
        title을 token 단위로 쪼갭니다.
        해당 코드는 prep_news.py에 있는 PrepNews 클래스의 process_sentence() 함수에서 가져왔습니다.
        """
        title_str: str = news['title']
        title_tokens = word_tokenize(title_str.strip().lower())

        idxs = []
        for token in title_tokens:
            idxs.append(word2int.loc[token, 'word_index'])
        title_idxs.append(torch.tensor(idxs, device=device))

    # Tensor 변환을 위해 마지막 차원의 길이는 모두 같아야 합니다.
    # 따라서 pad_sequence 함수를 이용해 가장 긴 길이에 맞춰 나머지에 padding을 추가합니다.
    title_tensor = pad_sequence(title_idxs, batch_first=True, padding_value=0)
    # Tensor의 device를 모델과 일치 시킵니다.
    title_tensor.to(device)
    # news vector를 얻습니다.
    news_vector: Tensor = model.news_encoder(title_tensor)

    return news_vector


In [24]:
"""
word embedding 레이어는 불러온 모델의 임베딩 레이어를 사용합니다.
해당 레이어는 전처리 과정에서 생성한 word2int와 인덱스가 연동되어있으므로
해당 파일을 불러와서 title을 index로 변환하겠습니다.
"""

model: NRMS = model_manager.model

word2int = model_manager.get_word2int()
word2int.set_index('word', inplace=True)
word2int.head()

,word_index
word,
the,1
brands,2
queen,3
elizabeth,4
",",5


In [27]:
candidate_news_vector = get_news_vector(candidate_news, model, word2int)
clicked_news_vector = get_news_vector(history_news, model, word2int)

print(candidate_news_vector.shape)
print(clicked_news_vector.shape)

torch.Size([2, 300])
torch.Size([2, 300])


In [28]:
# .unsqueeze(0)로 차원을 추가하여 shape를 (2, 300) -> (1, 2, 300)으로 변경합니다.
# 즉 입력 데이터를 batch 형태로 바꿔줍니다.
user_vector = model.user_encoder(clicked_news_vector.unsqueeze(0))
print(clicked_news_vector.unsqueeze(0).shape)
print(user_vector.shape)
print(user_vector[0].shape)

torch.Size([1, 2, 300])
torch.Size([1, 300])
torch.Size([300])


In [29]:
clicks_score = torch.bmm(
    candidate_news_vector.unsqueeze(0), # (1, 2, 300)
    user_vector.unsqueeze(dim=-1)       # (1, 300, 1) -> 직접 출력해보면 벡터의 숫자 300개를 한 칸씩 쪼개는 형태인 걸 알 수 있음
)
print(candidate_news_vector.unsqueeze(0).shape)
print(user_vector.unsqueeze(dim=-1).shape)
print(clicks_score.shape)
clicks_score = clicks_score.squeeze(dim=-1)
print(clicks_score.shape)
print(clicks_score)

torch.Size([1, 2, 300])
torch.Size([1, 300, 1])
torch.Size([1, 2, 1])
torch.Size([1, 2])
tensor([[ 30.2893, -15.2382]], device='cuda:0', grad_fn=<SqueezeBackward1>)


In [37]:
# 단순 내적과 결과 비교
a = torch.dot(candidate_news_vector[0], user_vector[0])
b = torch.dot(candidate_news_vector[1], user_vector[0])

print(f"[{a.item()}, {b.item()}] \n")

# 또는 이렇게도 가능
print(user_vector[0].shape)
print(candidate_news_vector.T.shape)
c = torch.matmul(user_vector[0], candidate_news_vector.T)
print(c.tolist())

[30.289268493652344, -15.238170623779297] 

torch.Size([300])
torch.Size([300, 2])
[30.289268493652344, -15.238171577453613]
